<a href="https://colab.research.google.com/github/Ajogeorge29/Medical_imaging_DICOM_Viewer/blob/main/Dicom2NiftiConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom nibabel numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00


In [ ]:
#DICOM(.dcm) files to Nifti (.nii) files

import os
import pydicom
import numpy as np
import nibabel as nib

def dicom_series_to_nifti(input_dir, output_file):
    """
    Convert a DICOM series to a NIfTI file.

    Parameters:
    - input_dir: Path to the directory containing DICOM files.
    - output_file: Path to the output NIfTI file.
    """

    # Get the list of all DICOM files in the input directory
    dicom_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.dcm')]

    if len(dicom_files) == 0:
        raise ValueError("No DICOM files found in the provided directory.")

    # Read the DICOM series
    dicom_series = [pydicom.dcmread(f) for f in dicom_files]

    # Sort the series based on Instance Number to maintain the correct order
    dicom_series.sort(key=lambda x: int(x.InstanceNumber))

    # Extract pixel data and stack them into a 3D numpy array
    pixel_arrays = [d.pixel_array for d in dicom_series]
    volume = np.stack(pixel_arrays, axis=-1)

    # Get pixel spacing and slice thickness from DICOM metadata
    pixel_spacing = dicom_series[0].PixelSpacing
    slice_thickness = dicom_series[0].SliceThickness

    # Define the affine transformation matrix
    affine = np.eye(4)
    affine[0, 0] = pixel_spacing[0]
    affine[1, 1] = pixel_spacing[1]
    affine[2, 2] = slice_thickness

    # Create a NIfTI image
    nifti_image = nib.Nifti1Image(volume, affine)

    # Save the NIfTI image
    nib.save(nifti_image, output_file)
    print(f"NIfTI file saved to {output_file}")

# Usage
input_directory = '/content'  # Replace with your DICOM directory path
output_nifti_file = 'trial1.nii'  # Replace with desired output file path

dicom_series_to_nifti(input_directory, output_nifti_file)



NIfTI file saved to trial1.nii


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# prompt: create code to visualise .nii file in python

import nibabel as nib
import matplotlib.pyplot as plt

def visualize_nifti(nifti_file):
  """
  Visualizes a NIfTI file using matplotlib.

  Args:
    nifti_file: Path to the NIfTI file.
  """

  img = nib.load(nifti_file)
  data = img.get_fdata()

  # Get the dimensions of the image
  x_dim, y_dim, z_dim = data.shape

  # Visualize a slice in the middle of the image in each dimension
  plt.figure(figsize=(15, 5))

  plt.subplot(1, 3, 1)
  plt.imshow(data[:, :, z_dim // 2], cmap='gray')
  plt.title('Axial Slice')

  plt.subplot(1, 3, 2)
  plt.imshow(data[:, y_dim // 2, :], cmap='gray')
  plt.title('Sagittal Slice')

  plt.subplot(1, 3, 3)
  plt.imshow(data[x_dim // 2, :, :], cmap='gray')
  plt.title('Coronal Slice')

  plt.show()


# Replace 'trial1.nii' with the path to your NIfTI file
visualize_nifti('trial1.nii')


In [ ]:
# prompt: the axis is not correctly oriented and also 3d visualisation is not seen

import os
import pydicom
import numpy as np
import nibabel as nib
from google.colab import drive
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
!pip install pydicom nibabel numpy


#DICOM(.dcm) files to Nifti (.nii) files


def dicom_series_to_nifti(input_dir, output_file):
    """
    Convert a DICOM series to a NIfTI file.

    Parameters:
    - input_dir: Path to the directory containing DICOM files.
    - output_file: Path to the output NIfTI file.
    """

    # Get the list of all DICOM files in the input directory
    dicom_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.dcm')]

    if len(dicom_files) == 0:
        raise ValueError("No DICOM files found in the provided directory.")

    # Read the DICOM series
    dicom_series = [pydicom.dcmread(f) for f in dicom_files]

    # Sort the series based on Instance Number to maintain the correct order
    dicom_series.sort(key=lambda x: int(x.InstanceNumber))

    # Extract pixel data and stack them into a 3D numpy array
    pixel_arrays = [d.pixel_array for d in dicom_series]
    volume = np.stack(pixel_arrays, axis=-1)

    # Get pixel spacing and slice thickness from DICOM metadata
    pixel_spacing = dicom_series[0].PixelSpacing
    slice_thickness = dicom_series[0].SliceThickness

    # Define the affine transformation matrix (adjusted for correct orientation)
    affine = np.array([
        [pixel_spacing[1], 0, 0, 0],
        [0, pixel_spacing[0], 0, 0],
        [0, 0, slice_thickness, 0],
        [0, 0, 0, 1]
    ])

    # Create a NIfTI image
    nifti_image = nib.Nifti1Image(volume, affine)

    # Save the NIfTI image
    nib.save(nifti_image, output_file)
    print(f"NIfTI file saved to {output_file}")

# Usage
input_directory = '/content'  # Replace with your DICOM directory path
output_nifti_file = 'trial1.nii'  # Replace with desired output file path

dicom_series_to_nifti(input_directory, output_nifti_file)


drive.mount('/content/drive')


def visualize_nifti(nifti_file):
  """
  Visualizes a NIfTI file using matplotlib.

  Args:
    nifti_file: Path to the NIfTI file.
  """

  img = nib.load(nifti_file)
  data = img.get_fdata()

  # Get the dimensions of the image
  x_dim, y_dim, z_dim = data.shape

  def plot_slice(z):
    plt.figure(figsize=(8, 8))
    plt.imshow(data[:, :, z], cmap='gray')
    plt.title(f'Axial Slice {z}')
    plt.show()

  interact(plot_slice, z=IntSlider(min=0, max=z_dim-1, step=1, value=z_dim//2))


# Replace 'trial1.nii' with the path to your NIfTI file
visualize_nifti('trial1.nii')


NIfTI file saved to trial1.nii
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


interactive(children=(IntSlider(value=335, description='z', max=669), Output()), _dom_classes=('widget-interac…

In [ ]:
# Otherway

!pip install pylibjpeg
!pip install pydicom nibabel numpy
import os
import pydicom
import numpy as np
import nibabel as nib

def dicom_series_to_nifti(input_dir, output_dir, output_file_name):
    """
    Convert a DICOM series to a NIfTI file and save it to a specified directory.

    Parameters:
    - input_dir: Path to the directory containing DICOM files.
    - output_dir: Path to the directory where the NIfTI file will be saved.
    - output_file_name: Name of the output NIfTI file.
    """

    # Ensure the output directory exists; if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created new directory: {output_dir}")

    # Get the list of all DICOM files in the input directory
    dicom_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.dcm')]

    if len(dicom_files) == 0:
        raise ValueError("No DICOM files found in the provided directory.")

    # Read the DICOM series
    dicom_series = [pydicom.dcmread(f) for f in dicom_files]

    # Sort the series based on Instance Number to maintain the correct order
    dicom_series.sort(key=lambda x: int(x.InstanceNumber))

    # Extract pixel data and stack them into a 3D numpy array
    pixel_arrays = [d.pixel_array for d in dicom_series]
    volume = np.stack(pixel_arrays, axis=-1)

    # Get pixel spacing and slice thickness from DICOM metadata
    pixel_spacing = dicom_series[0].PixelSpacing
    slice_thickness = dicom_series[0].SliceThickness

    # Define the affine transformation matrix
    affine = np.eye(4)
    affine[0, 0] = pixel_spacing[0]
    affine[1, 1] = pixel_spacing[1]
    affine[2, 2] = slice_thickness

    # Create a NIfTI image
    nifti_image = nib.Nifti1Image(volume, affine)

    # Construct the output file path
    output_file_path = os.path.join(output_dir, output_file_name)

    # Save the NIfTI image
    nib.save(nifti_image, output_file_path)
    print(f"NIfTI file saved to {output_file_path}")

# Usage
input_directory = '/content'  # Replace with your DICOM directory path
output_directory = '/content/output'     # Replace with desired output directory path
output_file_name = 'trial2.nii'          # Replace with desired output file name

dicom_series_to_nifti(input_directory, output_directory, output_file_name)


NIfTI file saved to /content/output/trial2.nii


In [ ]:
# prompt: how to visualise .nii in 3d in python

!pip install itkwidgets

import itkwidgets
import nibabel as nib

def visualize_nifti_3d(nifti_file):
  """
  Visualizes a NIfTI file in 3D using itkwidgets.

  Args:
    nifti_file: Path to the NIfTI file.
  """

  img = nib.load(nifti_file)
  data = img.get_fdata()

  # Create a 3D viewer using itkwidgets
  viewer = itkwidgets.view(data)
  return viewer


# Replace 'trial1.nii' with the path to your NIfTI file
viewer = visualize_nifti_3d('trial1.nii')
viewer


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…